<a href="https://colab.research.google.com/github/nice-bills/SnapValue/blob/main/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

url = "https://raw.githubusercontent.com/nice-bills/SnapValue/main/price-predictor/ebay_items_2023_present.csv"
df = pd.read_csv(url)
df.head(1)


,query,item_id,title,price,currency,condition,seller,feedback_score,image_url,item_url,item_creation_date
0,laptop,v1|396320750534|0,"CHUWI- 13.3"" Laptop - Intel Celeron 2.8GH- 8GB...",134.39,USD,New,chuwiofficial,413,https://i.ebayimg.com/images/g/ZysAAOSwXDpmIQc...,https://www.ebay.com/itm/396320750534?_skw=lap...,2025-03-15T01:18:10.000Z


In [4]:
import pandas as pd
from tqdm import tqdm
import json
from sentence_transformers import SentenceTransformer
import torch

In [6]:
CSV_FILE = "https://raw.githubusercontent.com/nice-bills/SnapValue/main/price-predictor/ebay_items_2023_present.csv"
OUTPUT_CSV = "ebay_items_with_text_embeddings.csv"
MODEL_NAME = 'all-MiniLM-L6-v2'

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:", device)
model = SentenceTransformer(MODEL_NAME, device=device)


Running on: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
df = pd.read_csv(CSV_FILE)
if "text_embedding" not in df.columns:
    df["text_embedding"] = None

n_skipped, n_success = 0, 0
text_embeddings = []

In [9]:
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating text embeddings"):
    text = str(row.get("title", "")).strip()
    if (not text or text.lower() in ["nan", "none"]) or (pd.notna(row.get("text_embedding")) and row.get("text_embedding")):
        text_embeddings.append(row.get("text_embedding") if pd.notna(row.get("text_embedding")) else None)
        n_skipped += 1
        continue
    try:
        emb = model.encode(text)
        text_embeddings.append(json.dumps(emb.tolist()))
        n_success += 1
    except Exception as e:
        print(f"Error embedding text '{text[:30]}...': {e}")
        text_embeddings.append(None)
        n_skipped += 1

df["text_embedding"] = text_embeddings
df.to_csv(OUTPUT_CSV, index=False)
print(f"\nSaved CSV with text embeddings: {OUTPUT_CSV}")
print(f"Embeddings written: {n_success} | Skipped: {n_skipped} | Total: {len(df)}")


Generating text embeddings: 100%|██████████| 10000/10000 [01:07<00:00, 148.99it/s]



Saved CSV with text embeddings: ebay_items_with_text_embeddings.csv
Embeddings written: 10000 | Skipped: 0 | Total: 10000


In [10]:
df = pd.read_csv("ebay_items_with_text_embeddings.csv")
df.head(1)


,query,item_id,title,price,currency,condition,seller,feedback_score,image_url,item_url,item_creation_date,text_embedding
0,laptop,v1|396320750534|0,"CHUWI- 13.3"" Laptop - Intel Celeron 2.8GH- 8GB...",134.39,USD,New,chuwiofficial,413,https://i.ebayimg.com/images/g/ZysAAOSwXDpmIQc...,https://www.ebay.com/itm/396320750534?_skw=lap...,2025-03-15T01:18:10.000Z,"[-0.06598962843418121, -0.025627294555306435, ..."


In [15]:
import pandas as pd
from pathlib import Path
import json
from tqdm import tqdm
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch

In [19]:
import pandas as pd
import requests
from pathlib import Path
from tqdm import tqdm
import mimetypes

In [20]:
CSV_FILE = "ebay_items_with_text_embeddings.csv"
IMAGE_FOLDER = Path("images")
IMAGE_FOLDER.mkdir(exist_ok=True)
OUTPUT_CSV = "ebay_items_with_image_paths.csv"

df = pd.read_csv(CSV_FILE)

def get_extension_from_url(url, response=None):
    ext = url.split(".")[-1].split("?")[0]
    if len(ext) > 5 or "/" in ext:
        if response is not None and "Content-Type" in response.headers:
            ext = mimetypes.guess_extension(response.headers["Content-Type"]) or ".jpg"
        else:
            ext = ".jpg"
    elif ext.lower() in ["jpeg", "jpg", "png", "webp"]:
        ext = "." + ext.lower()
    else:
        ext = ".jpg"
    return ext

def make_safe_filename(item_id, ext):
    safe_id = "".join(c if c.isalnum() else "_" for c in str(item_id))
    return safe_id + ext

def download_image(url, save_path):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            ext = get_extension_from_url(url, response)
            true_save_path = save_path.with_suffix(ext)
            with open(true_save_path, "wb") as f:
                f.write(response.content)
            return str(true_save_path)
    except Exception:
        pass
    return None

image_paths = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Downloading images"):
    url = str(row.get("image_url", ""))
    item_id = row.get("item_id", "missingid")

    if not url or pd.isna(url) or url.lower().strip() in ["none", "nan", ""]:
        image_paths.append(None)
        continue

    ext = get_extension_from_url(url)
    safe_filename = make_safe_filename(item_id, ext)
    save_path = IMAGE_FOLDER / safe_filename

    if save_path.exists():
        image_paths.append(str(save_path))
        continue

    result_path = download_image(url, save_path)
    image_paths.append(result_path)

df["image_path"] = image_paths
df.to_csv(OUTPUT_CSV, index=False)
print(f"Downloaded images and saved CSV to '{OUTPUT_CSV}'")


✅ Downloaded images and saved CSV to 'ebay_items_with_image_paths.csv'


In [21]:
df = pd.read_csv("ebay_items_with_image_paths.csv")
df.head(1)

,query,item_id,title,price,currency,condition,seller,feedback_score,image_url,item_url,item_creation_date,text_embedding,image_path
0,laptop,v1|396320750534|0,"CHUWI- 13.3"" Laptop - Intel Celeron 2.8GH- 8GB...",134.39,USD,New,chuwiofficial,413,https://i.ebayimg.com/images/g/ZysAAOSwXDpmIQc...,https://www.ebay.com/itm/396320750534?_skw=lap...,2025-03-15T01:18:10.000Z,"[-0.06598962843418121, -0.025627294555306435, ...",images/v1_396320750534_0.jpg


In [22]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from pathlib import Path
import pandas as pd
import json
from tqdm import tqdm

In [23]:
CSV_FILE = "ebay_items_with_image_paths.csv"
OUTPUT_JSON = "full_embeddings.json"
BATCH_SIZE = 32

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [25]:
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

In [26]:
df = pd.read_csv(CSV_FILE)
image_paths = df["image_path"].dropna().unique().tolist()

# === Generate embeddings ===
embeddings_dict = {}

for i in tqdm(range(0, len(image_paths), BATCH_SIZE), desc="Generating image embeddings"):
    batch_paths = image_paths[i:i+BATCH_SIZE]
    images = []
    valid_paths = []

    # Load images with error handling
    for path in batch_paths:
        try:
            img = Image.open(path).convert("RGB")
            images.append(img)
            valid_paths.append(path)
        except Exception as e:
            print(f"Skipping {path}: {e}")

    if not images:
        continue

    # Preprocess images and move tensors to device
    inputs = processor(images=images, return_tensors="pt").to(device)

    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
        # Normalize embeddings to unit length
        image_features = torch.nn.functional.normalize(image_features, p=2, dim=1)

    # Save embeddings keyed by file path
    for path, emb in zip(valid_paths, image_features.cpu().numpy()):
        embeddings_dict[path] = emb.tolist()

# === Save embeddings as JSON ===
with open(OUTPUT_JSON, "w") as f:
    json.dump(embeddings_dict, f)

print(f"\nSaved {len(embeddings_dict)} image embeddings to '{OUTPUT_JSON}'")


Generating image embeddings: 100%|██████████| 308/308 [01:00<00:00,  5.08it/s]



Saved 9841 image embeddings to 'full_embeddings.json'


In [27]:
import json

# === Config ===
CSV_FILE = "ebay_items_with_image_paths.csv"   # your CSV with 'image_path' column
EMBEDDINGS_FILE = "full_embeddings.json"       # JSON with image_path: embedding
OUTPUT_CSV = "ebay_items_with_full_embeddings.csv"

# === Load CSV and embeddings ===
df = pd.read_csv(CSV_FILE)

with open(EMBEDDINGS_FILE, "r") as f:
    embeddings_dict = json.load(f)

# === Map embeddings to CSV ===
image_embeddings = []

for path in df['image_path']:
    if pd.isna(path) or path not in embeddings_dict:
        image_embeddings.append(None)
    else:
        # Convert embedding list to comma-separated string
        emb_str = ",".join([str(x) for x in embeddings_dict[path]])
        image_embeddings.append(emb_str)

df['image_embedding'] = image_embeddings

# === Save updated CSV ===
df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ CSV saved with image embeddings: '{OUTPUT_CSV}'")


✅ CSV saved with image embeddings: 'ebay_items_with_full_embeddings.csv'


In [28]:
df = pd.read_csv("ebay_items_with_full_embeddings.csv")
df.head(1)

,query,item_id,title,price,currency,condition,seller,feedback_score,image_url,item_url,item_creation_date,text_embedding,image_path,image_embedding
0,laptop,v1|396320750534|0,"CHUWI- 13.3"" Laptop - Intel Celeron 2.8GH- 8GB...",134.39,USD,New,chuwiofficial,413,https://i.ebayimg.com/images/g/ZysAAOSwXDpmIQc...,https://www.ebay.com/itm/396320750534?_skw=lap...,2025-03-15T01:18:10.000Z,"[-0.06598962843418121, -0.025627294555306435, ...",images/v1_396320750534_0.jpg,"0.011063453741371632,0.05142253637313843,0.006..."
